In [2]:
import pandas as pd
import urllib.request
import re
import datetime
import itertools


In [23]:
def get_data(ur1):
    return str(urllib.request.urlopen(ur1).read())

def extract(data):
    result = data.split("underline")
    result = list(map(lambda x: x.split('</a><br/>')[0] , result))[1:]
    return result

def drop_bi(data):
    data = list(map(lambda x: re.sub('<b><i>(.*?)</i>', '<b>',x) , data))
    newdata = []
    for i in data:
        count = i.count('<br clear=left>')
        if count >= 2: #and 'provide alternate service' not in i:
            if 'provide alternate service' in i:
                newdata.append(i)
            else:
                j = i.split('<br clear=left>')
                for k in range(1, len(j)):
                    m = j[0] + j[k]
                    newdata.append(m)
        else:
            newdata.append(i)
    return newdata

def drop(data):
    data = list(map(lambda x: x.split('<b>')[1] , data))
    for i in data:
        count = i.count('<br clear=left>')
        if count > 1:
            i.split('<br clear=left>')
        
    return data

def rep(s):
    a= s.replace("<IMG src=\\'images/",' ').replace('6d', '6').replace(".png\\' >",'_LINE')
    
    return re.sub('<[^>]+>', '', a)

def rep_ls(ls):
    return list(map(rep, ls))

def drop_ff(ls, string):
    return list(map(lambda x: x.replace(string, ''), ls))

def clean(url):
    data = get_data(url)
    data = extract(data)
    data = drop_bi(data)
    data = drop(data)
    data = rep_ls(data)
    data = drop_ff(data,"F&F_Pos_16 (LINE)")
    data = drop_ff(data,"&nbsp;")
    data = drop_ff(data,"\\")
    return data

In [24]:
def get_clean_data(date):
    url = 'http://travel.mtanyct.info/serviceadvisory/routeStatusResult.aspx?tag=ALL&date='+date+'&time=&method=getstatus4'
    return clean(url)

In [25]:
def get_data_over_period(start, num_of_days):
    start_date = datetime.datetime.strptime(start, '%m/%d/20%y')
    date_list = [start_date + datetime.timedelta(days=x) for x in range(0, num_of_days)]
    dates = list(map(lambda x: x.strftime('%m/%d/20%y'), date_list))
    dataset = []
    dataset = list(map(get_clean_data, dates))
    answer = set(list(list(itertools.chain.from_iterable(dataset))))
    return answer

In [30]:
data = 'http://travel.mtanyct.info/serviceadvisory/routeStatusResult.aspx?tag=ALL&date=11/8/2017&time=&method=getstatus4'

In [28]:
get_clean_data('11/8/2017')

[' 1_LINE Van Cortlandt Park-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE Wakefield-bound trains skip 79 St and 86 St 1_LINE trains provide alternate service',
 ' 2_LINE Flatbush Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 2_LINE Wakefield-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE Flatbush Av-bound trains run local from 96 St to Times Sq-42 St',
 ' 3_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 3_LINE No trains running',
 ' 3_LINE New Lots Av-bound trains run local from 96 St to Times Sq-42 St',
 'No elevator service to/from the DOWNTOWN platform at Canal St  6_LINE Station',
 ' 4_LINE Woodlawn-bound trains skip 161 St, 167 St, 170 St, Mt Eden Av and 176 St',
 ' 4_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 4_LINE Trains run local in both directions between 125 St and Grand Central-42 St',
 ' 5

In [29]:
get_data_over_period('11/8/2017', 5)

{' 1_LINE Van Cortlandt Park-bound trains skip 50 St, 59 St and 66 St',
 ' 1_LINE Van Cortlandt Park-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE Flatbush Av-bound trains run local from 96 St to Times Sq-42 St',
 ' 2_LINE Flatbush Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 2_LINE Uptown trains skip 50 St, 59 St and 66 St',
 ' 2_LINE Wakefield-bound trains skip 79 St and 86 St 1_LINE trains provide alternate service',
 ' 2_LINE Wakefield-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE trains replace weekend  5_LINE service between Dyre Av and E 180 St',
 ' 3_LINE New Lots Av-bound trains run local from 96 St to Times Sq-42 St',
 ' 3_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 3_LINE No trains running',
 ' 4_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 4_LINE Service operates every 16 minutes between 